# ⚔️ Strategy Arena

Welcome to the Arena. Here we pit different trading strategies against each other to see who survives the market.

### Contenders:
1. **Buy & Hold**: The benchmark.
2. **Simple DCA**: Regular monthly investment.
3. **MA 200**: Classic trend following (Above=Buy, Below=Sell).
4. **David (Ladder)**: Blue Ladder Breakout/Breakdown.
5. **TQQQ DCA+**: Advanced cash management & profit taking.

---

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from core.strategies import BuyAndHold, SimpleDCA, MA200Strategy, DavidStrategy, TQQQ_DCA_Plus

# Cache
DATA_CACHE = {}

def get_data(ticker):
    if ticker not in DATA_CACHE:
        print(f"Fetching {ticker}...")
        # Get max history
        df = yf.download(ticker, period="max", interval="1d", progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            try:
                df.columns = df.columns.droplevel(1)
            except:
                pass
        # Filter to reasonable start date if needed, or keep full history
        DATA_CACHE[ticker] = df
    return DATA_CACHE[ticker].copy()

In [ ]:
def run_arena(ticker, initial_cash=100000):
    df = get_data(ticker)
    
    # Define Contenders
    strategies = [
        BuyAndHold(initial_cash),
        SimpleDCA(initial_cash, monthly_invest=1000),
        MA200Strategy(initial_cash),
        DavidStrategy(initial_cash),
        TQQQ_DCA_Plus(initial_cash)
    ]
    
    results = {}
    
    print(f"\nRunning simulations for {ticker}...")
    
    for strat in strategies:
        try:
            res_df = strat.run(df)
            
            # Metrics
            final_eq = res_df['Equity'].iloc[-1]
            ret = (final_eq - initial_cash) / initial_cash
            
            # Max Drawdown
            roll_max = res_df['Equity'].cummax()
            dd = (res_df['Equity'] - roll_max) / roll_max
            max_dd = dd.min()
            
            results[strat.name] = {
                'Equity': res_df['Equity'],
                'Return': ret,
                'MaxDD': max_dd,
                'Final': final_eq
            }
        except Exception as e:
            print(f"Strategy {strat.name} failed: {e}")
            
    # --- VISUALIZATION ---
    
    # 1. Performance Table
    metrics = pd.DataFrame(results).T[['Return', 'MaxDD', 'Final']]
    metrics['Return'] = metrics['Return'].apply(lambda x: f"{x:.2%}")
    metrics['MaxDD'] = metrics['MaxDD'].apply(lambda x: f"{x:.2%}")
    metrics['Final'] = metrics['Final'].apply(lambda x: f"${x:,.0f}")
    
    print("\n🏆 FINAL SCOREBOARD")
    display(metrics.sort_values('Final', ascending=False))
    
    # 2. Chart
    plt.figure(figsize=(14, 8))
    for name, data in results.items():
        plt.plot(data['Equity'], label=name, linewidth=2)
        
    plt.title(f"Strategy Comparison: {ticker}")
    plt.yscale('log')
    plt.ylabel('Portfolio Value (Log Scale)')
    plt.legend()
    plt.grid(True, which="both", ls="-", alpha=0.2)
    plt.show()

In [ ]:
# --- UI ---
w_ticker = widgets.Text(value='TQQQ', description='Ticker:')
w_btn = widgets.Button(description='FIGHT! ⚔️', button_style='danger')

out = widgets.Output()

def on_click(b):
    with out:
        from IPython.display import clear_output
        clear_output(wait=True)
        run_arena(w_ticker.value)

w_btn.on_click(on_click)
display(widgets.HBox([w_ticker, w_btn]))
display(out)